In [2]:
import os
import librosa                                       # 음성 분석을 위함
import threading                                     # 병렬처리를 위한 스레딩 모듈
import numpy as np                                      
import soundfile as sf
import tensorflow as tf
import tensorflow_hub as hub
import scipy.io.wavfile as wav
from pydub.playback import play
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from pydub import AudioSegment                       # 오디오를 처리를 위함
import matplotlib.pyplot as plt                      # 그래프를 그리기 위함
from scipy.interpolate import interp1d
model = hub.load("https://tfhub.dev/google/spice/2") # SPICE 모델 로드

## 1. 음정 주파수 추출 함수 정의

In [3]:
EXPECTED_SAMPLE_RATE = 16000                                                                    # 지정할 Sample rate

def extract_pitches(audio_path, duration):
    original_y, sr = librosa.load(audio_path, mono=True, duration=duration)                     # SPICE모델은 mono 타입의 오디오만 지원

    MAX_ABS_INT16 = 32768.0                                                                     # 16비트 PCM 형식에서 가능한 최대 절댓값을 나타내는 상수

    y = original_y/float(MAX_ABS_INT16)                                                         # -1과 1사이의 부동 소수점으로 정규화

    model_output = model.signatures["serving_default"](tf.constant(y, tf.float32))              # 음정과 신뢰도를 얻음

    pitch_outputs = model_output["pitch"].numpy()                                               # 음정만 추출
    
    original_duration = len(y) / sr                                                             # 원래 신호의 전체 시간 (초)

    pitch_time = np.linspace(0, original_duration, len(pitch_outputs))                          # 음정 출력의 시간 축 생성 (초 단위)
    resampled_time = np.linspace(0, original_duration, int(original_duration))                  # 원곡 시간 축 생성 (초 단위)

    interp_func = interp1d(pitch_time, pitch_outputs, kind='linear', fill_value="extrapolate")  # 보간 함수 생성
    
    resampled_pitch_outputs = interp_func(resampled_time)                                       # 보간법 적용하여 음정 출력을 원래 신호의 길이에 맞춤

    file_path = audio_path.replace('.wav', '.npy')                                              # 오디오 파일명과 같은 이름으로 npy 파일 생성

    np.save(file_path, resampled_pitch_outputs)                                                 # 오디오 배열 저장

    return original_y, resampled_pitch_outputs, sr

# 2. 사용자 음성 받기

In [4]:
audio_name = '장범준-흔들리는 꽃들 속에서 네 샴푸향이 느껴진거야.wav'

artist_audio_path = f'assets/assets/audio/artist/{audio_name}'          # 원곡 음원 경로
user_audio_path = f'assets/assets/audio/user/{audio_name}'              # 커버 음원 경로

In [6]:
# 음원 파일 재생 함수
def play_audio(audio_file):
    song = AudioSegment.from_file(audio_file) # 오디오 파일 불러오기
    play(song)                                # 오디오 파일 재생

# 사용자 음성 녹음 함수
def record_audio(duration, output_file, sync_delay=0):
    fs = 44100                                # 샘플링 레이트
    print("녹음 시작")
    recording = sd.rec(int((duration + sync_delay) * fs), samplerate=fs, channels=2, dtype='int16')
    sd.wait()                                 # 녹음이 끝날 때까지 대기
    print("녹음 끝")
    
    # 지연 시간만큼 녹음된 파일에서 잘라내기
    if sync_delay > 0:
        recording = recording[int(sync_delay * fs):]
    
    wav.write(output_file, fs, recording)     # 녹음 저장(파일명, 샘플링 레이트, 음성 파일)
    
def record_and_play(artist_audio_path):

    artist_audio = f'{artist_audio_path}.wav'                # 
    user_audio_path = f'user_audio/{audio_name}-cover.wav'   # 녹음된 사용자 음성을 저장할 경로
    sync_delay = 0.4                                         # 음성 녹음 지연 시간 (초 단위)
    
    # 음원 파일 길이 가져오기
    song = AudioSegment.from_file(artist_audio) # 음원 파일 불러오기
    duration = len(song) / 1000.0               # 밀리초를 초 단위로 변환

    # 스레드를 사용하여 동시에 재생 및 녹음 실행
    playback_thread = threading.Thread(target=play_audio, args=(artist_audio,))                           # 스레드 생성(함수와 인자)
    recording_thread = threading.Thread(target=record_audio, args=(duration, user_audio_path, sync_delay))

    # 녹음과 재생을 동시에 시작
    recording_thread.start()                   # 스레드 시작 (녹음)
    playback_thread.start()                    # 스레드 시작 (재생)

    playback_thread.join()                     # 스레드 종료 대기 (재생)
    recording_thread.join()                    # 스레드 종료 대기 (녹음)

    print(f"녹음 저장 완료: {user_audio_path}")

record_and_play(artist_audio_path)

녹음 시작
녹음 끝
녹음 저장 완료: user_audio/flower-cover.wav


In [ ]:
audio_name = 'flower'                                                               # 노래 제목

artist_audio_path = f'artist_audio/{audio_name}'                                    # 원곡 음원 경로

user_audio_path = 'user_recording'                                                  # 커버 음원 경로

y, sr =  librosa.load(f'{artist_audio_path}.wav', mono=True)
duration = len(y)/sr

artist_audio, resampled_artist_audio , sr = extract_pitches(f'{artist_audio_path}.wav', duration)
user_audio, resampled_user_audio, _ = extract_pitches(f'{user_audio_path}.wav', duration)